In [11]:
from discord_notify import send_discord_notification
from gemini import get_api_key

In [12]:
send_discord_notification("Start Quantum Circuit Artificail Intelligence")

In [13]:
GEMINI_API_KEY = get_api_key()

In [14]:
import time

In [15]:
def time_logger(original_function):
    def wrapper(*args, **kwargs):
        process_start_time = time.process_time()
        performance_start_time = time.perf_counter()

        result = original_function(*args, **kwargs)
        
        process_end_time = time.process_time()
        performance_end_time = time.perf_counter()
        execution_time = process_end_time - process_start_time
        performance_time = performance_end_time - performance_start_time
        print(f"Execution time: {execution_time:.6f} seconds", end=", ")
        print(f"Performance time: {performance_time:.6f} seconds")
        send_discord_notification(f"Execution time: {execution_time:.6f} seconds, Performance time: {performance_time:.6f} seconds")
        return result
    return wrapper

In [16]:
def Update_Model_List(_model_name, _prompt, _model, _chain_list):
    print(f"Updating model list with model: {_model_name}")
    global models_list
    if _model_name not in models_list:
        models_list.append(_model_name)
    _temp = dict()
    _temp[_model_name] = _prompt | _model
    _chain_list.append(_temp)

In [17]:
"""
def Get_Answer_caller(_chain_list, query):
    _answer_list = list()
    for models in _chain_list:
        for name, model in models.items():
            print(name, end=": ")
            process_start_time = time.process_time()
            performance_start_time = time.perf_counter()

            _answer_list.append({name : model.invoke({"question": query})})

            process_end_time = time.process_time()
            performance_end_time = time.perf_counter()
            execution_time = process_end_time - process_start_time
            performance_time = performance_end_time - performance_start_time
            print(f"Execution time: {execution_time:.4f} seconds", end=", ")
            print(f"Performance time: {performance_time:.4f} seconds")
    return _answer_list
"""

def Get_Answer_caller(query):
    _query = query
    @time_logger
    def Get_Answer(model):
        return model.invoke({"question": _query})
    return Get_Answer

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
import ollama
from langchain_google_genai import GoogleGenerativeAI

chain_list = list()
model_dict = dict()
models_list = list()
for _ in ollama.list().models:
    models_list.append(_["model"])

template = """Question: {question}

Answer: Answer the question in Korean."""

prompt = ChatPromptTemplate.from_template(template)

for model_name in models_list:
    model = OllamaLLM(model=model_name)
    Update_Model_List(model_name, prompt, model, chain_list)

llm = GoogleGenerativeAI(model="gemini-2.5-pro-exp-03-25", google_api_key=GEMINI_API_KEY)
Update_Model_List("Gemini-2.5-pro", prompt, llm, chain_list)

print("Models List:")
print(", ".join(models_list))
send_discord_notification(f"Models List: {', '.join(models_list)}")

Updating model list with model: deepseek-r1:70b
Updating model list with model: gemma3:27b
Updating model list with model: llama3.3:70b
Updating model list with model: llama3.1:70b
Updating model list with model: Gemini-2.5-pro
Models List:
deepseek-r1:70b, gemma3:27b, llama3.3:70b, llama3.1:70b, Gemini-2.5-pro


In [19]:
answer_list = list()
answer_caller = Get_Answer_caller("OpenQASM 형태의 양자 컴퓨팅 회로를 설계할 수 있는가? 설계할 수 있다면, 양자 회로를 설계하는 방법을 설명 및 예시를 제시하라.")
try:
    for models in chain_list:
        for name, model in models.items():
            print(name, end=": ")
            send_discord_notification(f"Start {name}")
            answer = answer_caller(model)
            send_discord_notification(f"End {name}")
            answer_list.append({name: answer})
            # print(name, end=": ")
            # print(answer)
            # send_discord_notification(f"{name}: {answer}")
            
except Exception as e:
    print(f"Error occurred: {e}")
    send_discord_notification(f"Error occurred: {e}")

deepseek-r1:70b: Execution time: 0.277493 seconds, Performance time: 139.595303 seconds
gemma3:27b: Execution time: 0.374473 seconds, Performance time: 81.185700 seconds
llama3.3:70b: Execution time: 0.123702 seconds, Performance time: 71.372826 seconds
llama3.1:70b: Execution time: 0.211368 seconds, Performance time: 88.645144 seconds
Gemini-2.5-pro: Execution time: 0.036201 seconds, Performance time: 30.978114 seconds


In [21]:
for _ in answer_list:
    for name, answer in _.items():
        print(name, end=": ")
        print(answer+"\n")
        # send_discord_notification(f"{name}: {answer}")

deepseek-r1:70b: <think>
Okay, so I need to figure out how to design a quantum circuit using OpenQASM. I'm not super familiar with this, but I'll try to break it down step by step.

First, I remember that QASM stands for Quantum Assembly Language. It's like a low-level programming language for quantum computers. So OpenQASM must be an open-source version or implementation of that. 

I think the basic structure involves defining qubits and operations on them. Maybe I need to start with a header that specifies the version, like OPENQASM 1.0; or something similar. Then, I have to declare how many quantum registers (qubits) and classical registers (for measurements) I'll use. 

Let me think about a simple example. Suppose I want to create a circuit that applies a Hadamard gate to the first qubit and then measures it. How would that look in OpenQASM?

I'd probably start with something like:
OPENQASM 1.0;
qubit[2] q; // Maybe I need two qubits? Or just one for this case.
cx q[0], q[1]; // Wa

# Verify Generated Quantum Circuit

In [23]:
# Load the Qiskit Runtime service
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService()

In [24]:
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler

# 1. A quantum circuit for preparing the quantum state (|00> + |11>)/rt{2}
bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0, 1)
bell.measure_all()

# 2: Optimize problem for quantum execution.
backend = service.least_busy(operational=True, simulator=False)
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(bell)

# 3. Execute using the Sampler primitive
sampler = Sampler(mode=backend)
sampler.options.default_shots = 1024  # Options can be set using auto-complete.
job = sampler.run([isa_circuit])
print(f"Job ID is {job.job_id()}")
pub_result = job.result()[0]
print(f"Counts for the meas output register: {pub_result.data.meas.get_counts()}")

Job ID is czsk2pjkzhn0008c8wn0
Counts for the meas output register: {'10': 88, '11': 438, '00': 415, '01': 83}
